# DTLZ 2:
Here we show that RVEA works better if we have limited function evaluations (5000 in this case) for dtlz .

# DTLZ 7
Here we show that even tho NSGAIII finds more solutions the few solutions that RVEA finds are better (hopefully have not checked numbers yet)

## Optimization process


In [32]:
import numpy as np
import pandas as pd
import math


import random
import scipy

from desdeo_tools.utilities.pmod import get_pmod
from desdeo_tools.utilities.pmda import get_pmda
import ehmetric as eh
from desdeo_tools.utilities.rmetric import RMetric



from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from desdeo_problem import variable_builder, MOProblem, VectorObjective
from desdeo_problem.testproblems.TestProblems import test_problem_builder
from sklearn.preprocessing import Normalizer
from desdeo_emo.EAs import NSGAIII, RVEA, IOPIS_RVEA, IOPIS_NSGAIII, IBEA

from desdeo_emo.utilities import animate_init_, animate_next_

refpoints_dtlz2 = [[0.10, 0.10, 0.94],[0.10, 0.94, 0.10],  [0.94, 0.10, 0.10], [0.7,0.6,0.55] ]
refpoints_dtlz7 = [[0.70, 0.10, 4.00 ],[0.06, 0.10, 3.2],  [0.05, 0.70, 3.5], [0.80, 0.80, 2.60] ]

refpoints = refpoints_dtlz2
minimization = [True]*3 # for eh-metric
rvea_solutions = []
nsga_solutions = []
RP = []
n_interactions = 4
n_obj = 3
n_var = 8
name = "DTLZ2"
problem = test_problem_builder(
    name=name, n_of_objectives=n_obj, n_of_variables=n_var
    )
evolver_rvea = RVEA(problem, n_gen_per_iter=50, n_iterations=1, interact=True )#total_function_evaluations= 9000000
evolver_nsga = NSGAIII(problem, n_gen_per_iter=50, n_iterations=1, interact=True) #total_function_evaluations= 600000
#evolver_nsga.translation_param = 0.3

#evolver_nsga = IOPIS_RVEA(problem, n_gen_per_iter=500, n_iterations=1)
problem.ideal = np.asarray([0] * n_obj)
problem.ideal_fitness = problem.ideal
problem.nadir = abs(np.random.normal(size=n_obj, scale=0.15)) + 1

non_dominated = evolver_rvea.population.non_dominated_fitness()

for i in range(n_interactions):
    pref_rvea, plot_rvea = evolver_rvea.requests()
    pref_rvea[2].response = pd.DataFrame(
    [refpoints[i]], columns=pref_rvea[2].content["dimensions_data"].columns
    )
    pref_nsga, plot_nsga = evolver_nsga.requests()
    pref_nsga[2].response = pd.DataFrame(
    [refpoints[i]], columns=pref_nsga[2].content["dimensions_data"].columns
    )
    print(f"Running iteration {evolver_rvea._iteration_counter+1}")
    evolver_rvea.iterate(pref_rvea[2])
    evolver_nsga.iterate(pref_nsga[2]) #

    non_dominated_rvea = evolver_rvea.population.non_dominated_fitness()
    rvea_results = evolver_rvea.population.objectives[non_dominated_rvea]
    rvea_solutions.append(rvea_results)

    non_dominated_nsga = evolver_nsga.population.non_dominated_fitness()
    nsga_results = evolver_nsga.population.objectives[non_dominated_nsga]
    nsga_solutions.append(nsga_results)
    
    


Running iteration 1
Running iteration 2
Running iteration 3
Running iteration 4


# Visualization

In [33]:
figure_rvea = animate_init_(np.vstack((rvea_solutions[0],[1,1,1],[0,0,0])), filename="RVEAw"+name+".html")
figure_nsga = animate_init_(np.vstack((nsga_solutions[0],[1,1,1],[0,0,0])), filename="NSGA"+name+".html")


for i in range(n_interactions-1):
    figure_rvea = animate_next_(
            np.vstack((rvea_solutions[i+1], [1,1,1],[0,0,0])),
            figure_rvea,
            filename="RVEAw"+name+".html",
            generation=i+1,
        )
    figure_nsga = animate_next_(
            np.vstack((nsga_solutions[i+1], [1,1,1],[0,0,0])),
            figure_nsga,
            filename="NSGA"+name+".html",
            generation=i+1,
        )

Plot saved as:  RVEAwDTLZ2.html
View the plot by opening the file in browser.
To view the plot in Jupyter Notebook, use the IFrame command.
Plot saved as:  NSGADTLZ2.html
View the plot by opening the file in browser.
To view the plot in Jupyter Notebook, use the IFrame command.


# Calculating some indicators


In [37]:
for i in range(n_interactions):
    

    print("this is [RVEA, NSGA] pmod:", [get_pmod(refpoints[i],rvea_solutions[i],0.1,2), get_pmod(refpoints[i],nsga_solutions[i],0.1,2)])
    print("this is [RVEA, NSGA] pmda:", [get_pmda(refpoints[i],rvea_solutions[i],3,0.5,0.01), get_pmda(refpoints[i],nsga_solutions[i],3,0.5,0.01)])
    

    #R-metric EH-metric
    #normalization
    _max = np.max(np.vstack((rvea_solutions[i],nsga_solutions[i], refpoints[i])) , axis= 0)
    _min = np.min(np.vstack((rvea_solutions[i],nsga_solutions[i], refpoints[i])) , axis= 0)
    norm_rvea = (rvea_solutions[i] - _min)/ (_max - _min)
    norm_nsga = (nsga_solutions[i] - _min)/ (_max - _min)
    norm_RP = (refpoints[i] - _min)/ (_max - _min)

    areas, steps = eh.RunEHMetric([pd.DataFrame(norm_rvea), pd.DataFrame(norm_nsga)], norm_RP, minimization)
    print("this is [RVEA, NSGA] EH-metric:", areas)
    rmetric_rvea = RMetric(problem, norm_RP, delta =0.3).calc(F = norm_rvea, others = norm_nsga)
    rmetric_nsga = RMetric(problem, norm_RP, delta = 0.3).calc(F = norm_nsga, others = norm_rvea)
    print("this is [RVEA, NSGA] R-metric:", [rmetric_rvea[1], rmetric_nsga[1]])


this is [RVEA, NSGA] pmod: [20.32474310828537, 30.745720803336923]
this is [RVEA, NSGA] pmda: [0.620384007326949, 0.5994900174078085]
this is [RVEA, NSGA] EH-metric: [0.7732071579510699, 0.8118029984628999]
this is [RVEA, NSGA] R-metric: [8.411764954586706, 8.744689373406386]
this is [RVEA, NSGA] pmod: [23.864017000310465, 35.26963913911691]
this is [RVEA, NSGA] pmda: [0.6145247659335346, 0.5985656522742909]
this is [RVEA, NSGA] EH-metric: [0.7505302026368251, 0.8165223252438842]
this is [RVEA, NSGA] R-metric: [7.94805356353462, 8.684241398268828]
this is [RVEA, NSGA] pmod: [21.90456117055672, 38.370943919524244]
this is [RVEA, NSGA] pmda: [0.6142232070280055, 0.5980365716447347]
this is [RVEA, NSGA] EH-metric: [0.7602276448561636, 0.8148339344456919]
this is [RVEA, NSGA] R-metric: [7.959376432223174, 8.762577659074548]
this is [RVEA, NSGA] pmod: [31.3868615711101, 49.665205848363534]
this is [RVEA, NSGA] pmda: [0.7316035228151493, 0.7156954937713653]
this is [RVEA, NSGA] EH-metric: [0

In [31]:
norm_rvea

array([[0.11660326, 0.94001116, 0.30546229],
       [0.20227659, 0.95008927, 0.23006777],
       [0.1709114 , 0.94746536, 0.25427529],
       [0.25013579, 0.93242009, 0.24964023],
       [0.10276323, 0.94062322, 0.32186516],
       [0.23021089, 0.92072337, 0.31043982],
       [0.14014855, 0.96852254, 0.20076512],
       [0.16689752, 0.93906766, 0.2857815 ],
       [0.2811834 , 0.9349719 , 0.19317044],
       [0.20243487, 0.94446421, 0.26067954],
       [0.29915434, 0.93229692, 0.22564325],
       [0.13479839, 0.94942163, 0.26939343],
       [0.2514848 , 0.92993215, 0.28168986],
       [0.16511654, 0.93543466, 0.30955145],
       [0.31775509, 0.92839894, 0.16526159],
       [0.30976647, 0.95541873, 0.1031505 ],
       [0.10213517, 0.93223077, 0.355531  ],
       [0.19566987, 0.93527923, 0.31578438],
       [0.23749029, 0.95491308, 0.16775929],
       [0.11579021, 0.97718083, 0.13630432],
       [0.13140146, 0.98668057, 0.01581186],
       [0.08139509, 0.95668904, 0.27025694],
       [0.

## PMOD

## PMDA

ModuleNotFoundError: No module named 'bokeh'